In [3]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from os import path

os.getcwd()

'/home/matta/recipy_maker/Web_Scraping'

In [40]:
MAIN_STRING = "carouselNav__list elementFont__resetList recipeCarousel__list"  # class for main recipe category page navlist
CAT_STRING = "category-page-list-content category-page-list-content__recipe-card karma-main-column"  # Class for category page div
CARDS_STRING = "card__titleLink manual-link-behavior elementFont__title margin-8-bottom"  # Class for most popular recipe cards

def get_links():
    if path.exists('list_w_img.csv'):
        df = pd.read_csv('list_w_img.csv')
    else:
        source = requests.get('https://www.allrecipes.com/recipes/').text
        soup = BeautifulSoup(source, 'lxml')
        recipe_list = soup.find('ul', class_=MAIN_STRING)
        links = [a['href'] for a in recipe_list.find_all('a', href=True)]
        pop_recipes = []
        for link in links:
            category = link.split('/')[5]
            recipe_category = requests.get(link).text
            cat_soup = BeautifulSoup(recipe_category, 'lxml')  # category page (cat_soup)
            category_lists = cat_soup.find_all('div', class_=CAT_STRING)  
            for lst in category_lists:
                for a in lst.find_all('a', class_=CARDS_STRING, href=True):
                    if re.search('https://www.allrecipes.com/recipe/.*', a['href']):
                        pop_recipes.append([a['href'], a.div['data-src'],category])
        


        df = pd.DataFrame(pop_recipes, columns=["link","image_url","category"])
        df.to_csv('list_w_img.csv', index=False)
    return df

In [41]:
df = get_links()
df = df.drop_duplicates()
df.shape


(659, 3)

In [26]:
# Runs ~2 minutes

recipe_info = []
for index,row in df.iterrows():
    link = row['link']
    image_url = row['image_url']
    source = requests.get(link).text
    soup = BeautifulSoup(source, 'lxml')
    try:
        title = soup.find('h1').text
        description = soup.find('p').text
        rating = soup.find('span',class_="review-star-text visually-hidden").text

        ingredient_list = soup.find('ul',class_ = "ingredients-section")
        ingredients = []
        for ingredient in ingredient_list.find_all('span',class_="ingredients-item-name elementFont__body"):
            ingredients.append(ingredient.text)

        instruction_list = soup.find('ul',class_ = "instructions-section")
        instructions = []
        for instruction in instruction_list.find_all('p'):
            instructions.append(instruction.text)
        
        recipe_info.append([link,title,image_url,description,rating,ingredients,instructions])
    except:
        continue

df2 =  pd.DataFrame(recipe_info, columns=["link","title","image_url","description","rating","ingredients","instructions"])


In [ ]:
df2

In [20]:
df2.to_pickle('full_data_with_image.pkl')